# CO<sub>2</sub> Prediction using Facebook Prophet

<div>
<img src="https://drive.google.com/uc?id=1YxXGZbFQ5qsjXuCGOzVXzx97GuLoWGp2" width="800"/>
</div>

### Facebook Prophet on-line documentation can be found at:

**https://facebook.github.io/prophet/docs/quick_start.html**

## 1. Importing the data and plotting it

In [ ]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 70)

plt.rcParams['figure.figsize'] = (20, 10)

data = sm.datasets.co2.load_pandas()
co2 = data.data
co2 = co2.dropna()
ax = co2.plot()
ax.set_ylabel('CO2 level')
display(plt.show())

## 2. Clean the data, create datasets for testing and training

In [ ]:
data.data.tail()

In [ ]:
y=co2.reset_index(drop=False)
#y.columns = ['ds', 'y']
y = y.rename(columns={'index':'ds','co2':'y'})
y.head()

In [ ]:
y.info()

In [ ]:
train = y.iloc[:2000,:]
test = y.iloc[2000:,:]

## 3. Train Prophet model

In [ ]:
if 'google.colab' in str(get_ipython()):
    from fbprophet import Prophet
else:
    from prophet import Prophet

import os
# Define a context manager to suppress stdout and stderr.
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds =  [os.open(os.devnull,os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = [os.dup(1), os.dup(2)]

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0],1)
        os.dup2(self.null_fds[1],2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0],1)
        os.dup2(self.save_fds[1],2)
        # Close all file descriptors
        for fd in self.null_fds + self.save_fds:
            os.close(fd)

with suppress_stdout_stderr():
    m = Prophet(interval_width=0.95)
    m.fit(train)

## 4. Run prediction using Prophet model above

In [ ]:
#future = m.make_future_dataframe(periods=len(test), freq="W")
future = y.copy()
future.drop('y', inplace=True, axis=1)
forecast = m.predict(future)

In [ ]:
forecast.head()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(list(test['y']), list(forecast.loc[2000:,'yhat'] )))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20, 10)
plt.plot(list(test['y']), 'red')
plt.plot(list(forecast.loc[2000:,'yhat']), "cyan")
plt.show()

In [ ]:
if 'google.colab' in str(get_ipython()):
    from fbprophet.plot import add_changepoints_to_plot
else:
    from prophet.plot import add_changepoints_to_plot
    
#from prophet.plot import plot_plotly, plot_components_plotly

#plot_components_plotly(m, forecast)
fig1 = m.plot(forecast,  figsize=(20, 12))
a = add_changepoints_to_plot(fig1.gca(), m, forecast)

In [ ]:
fig2 = m.plot_components(forecast, figsize=(20, 12))
plt.show()

## 5. Incorporate monthly seasonality

In [ ]:
m2 = Prophet(interval_width=0.95)
m2.add_seasonality(name='monthly', period=30.5, fourier_order=5)

with suppress_stdout_stderr():
    m2.fit(train)

#future2 = m2.make_future_dataframe(periods=len(test), freq="W")
forecast2 = m2.predict(future)
print(r2_score(list(test['y']), list(forecast2.loc[2000:,'yhat'] )))

fig2 = m2.plot_components(forecast2, figsize=(20, 12))
plt.show()

## 6. Add regressors to allow for autoregression

In [ ]:
yr = y.copy()
yr['co2_l1']=yr['y'].shift(1)
yr['co2_l2']=yr['y'].shift(2)
yr['co2_l3']=yr['y'].shift(3)
yr['co2_l4']=yr['y'].shift(4)
yr['co2_l5']=yr['y'].shift(5)
yr = yr.dropna()
yr=yr.reset_index(drop=True)
yr.head()

In [ ]:
train3 = yr.iloc[:2000,:]
test3 = yr.iloc[2000:,:]

In [ ]:
m3 = Prophet(interval_width=0.95)
m3.add_regressor('co2_l1')
m3.add_regressor('co2_l2')
m3.add_regressor('co2_l3')
m3.add_regressor('co2_l4')
m3.add_regressor('co2_l5')

with suppress_stdout_stderr():
    m3.fit(train3)

future3 = yr.copy()
future3.drop('y', inplace=True, axis=1)

forecast3 = m3.predict(future3)
forecast3.tail()

In [ ]:
print(r2_score(list(test3['y']), list(forecast3.loc[2000:,'yhat'] )))

plt.plot(list(test3['y']), "orange")
plt.plot(list(forecast3.loc[2000:,'yhat'] ))

fig3 = m3.plot_components(forecast3, figsize=(20, 12))
plt.show()


In [ ]:
fig3 = m3.plot(forecast3,  figsize=(20, 12))
a3 = add_changepoints_to_plot(fig3.gca(), m3, forecast3)